# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
df = pd.read_csv('cityweather.csv')
dfcopy = df.copy()

df.head()

,Cities,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature (K),Wind Speed
0,Rikitea,0,PF,1569455467,63,-23.12,-134.97,295.435,2.05
1,Nikolskoye,90,RU,1569455467,95,59.70,30.79,276.480,2.00
2,Grand Gaube,20,MU,1569455467,94,-20.01,57.66,294.260,2.10
3,Ferme-Neuve,100,CA,1569455468,87,46.70,-75.45,287.645,5.06
4,Palmer,0,AU,1569455174,39,-34.85,139.16,294.820,4.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
gmaps.configure(api_key = g_key)

lat_long = dfcopy[['Latitude','Longitude']]
humidity = dfcopy['Humidity'].astype(float)

figure1 = gmaps.figure()
heatmap = gmaps.heatmap_layer(lat_long, weights = humidity,
                              dissipating=False, max_intensity=100,point_radius=3)

figure1.add_layer(heatmap)

figure1



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
# Since the temps are in kelvin, 60 degree farhenheit is 289K and 80 degree farhenheit is 300K.

refined_df = dfcopy.loc[(dfcopy['Max Temperature (K)'] > 289 ) & (dfcopy['Max Temperature (K)']
                       < 300) & (dfcopy['Cloudiness'] == 0.0) & (dfcopy['Wind Speed'] <= 10)]

refined_df = refined_df.dropna()
refined_df.count()

Cities                 45
Cloudiness             45
Country                45
Date                   45
Humidity               45
Latitude               45
Longitude              45
Max Temperature (K)    45
Wind Speed             45
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [88]:
hotel_df = refined_df

hotel_df = hotel_df.reset_index()

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotel_df['Hotel Names'] = [x for x in range(len(hotel_df['Humidity']))]
hotel_df['Hotel Coordinates'] = [x for x in range(len(hotel_df['Humidity']))]
hotel_df['Hotel Latitude'] = [x for x in range(len(hotel_df['Humidity']))]
hotel_df['Hotel Longitude'] = [x for x in range(len(hotel_df['Humidity']))]


results = []

for x in range(len(hotel_df['Humidity'])):
    
    target_coordinates = str(hotel_df.iloc[x,6]) + ', ' + str(hotel_df.iloc[x,7])
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

#     print(f"params entered iteration:{x}")

    params = {

            "location": target_coordinates,
            "keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": g_key

    }

    response = requests.get(base_url, params=params)

    data = response.json()
    
    results.append(data)
    
    try:
        hotelname = data['results'][0]['name']
        coord_lat = data['results'][0]['geometry']['location']['lat']
        coord_lng = data['results'][0]['geometry']['location']['lng']
        coord = str(coord_lat) + ', ' + str(coord_lng)
        
        hotel_df.iloc[x, 10] = hotelname
        hotel_df.iloc[x, 11] = coord
        hotel_df.iloc[x, 12] = coord_lat
        hotel_df.iloc[x, 13] = coord_lng

        
    except IndexError:
        hotel_df.iloc[x, 10] = 'No Results'
        hotel_df.iloc[x, 11] = 'No Results'
        hotel_df.iloc[x, 12] = 'No Results'
        hotel_df.iloc[x, 13] = 'No Results'

   
    #print(f"Hotel entered, iteration: {x}")

hotel_df

# test = results[0]

# coord = test[0]['geometry']['location']

# # pprint(test)

# print(coord)



# Notes: Error at index 18, no results, we need to account for this.


# print(data)



# pprint(data, depth = 3)

# target_coordinates


# hotel_df

,index,Cities,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature (K),Wind Speed,Hotel Names,Hotel Coordinates,Hotel Latitude,Hotel Longitude
0,0,Rikitea,0,PF,1569455467,63,-23.12,-134.97,295.435,2.05,Pension Bianca & Benoit,"-23.1276524, -134.9656596",-23.1277,-134.966
1,4,Palmer,0,AU,1569455174,39,-34.85,139.16,294.820,4.60,Palmer Hotel,"-34.8521749, 139.1597109",-34.8522,139.16
2,6,Hermanus,0,ZA,1569455468,65,-34.42,19.24,291.480,4.47,Windsor Hotel,"-34.4212214, 19.2401906",-34.4212,19.2402
3,7,Turayf,0,SA,1569455468,37,31.68,38.65,295.150,5.70,Golden Dune Hotel,"31.67244639999999, 38.6596733",31.6724,38.6597
4,35,Cape Town,0,ZA,1569455359,87,-33.93,18.42,291.480,2.10,Mojo Hotel,"-33.919248, 18.3859406",-33.9192,18.3859
5,56,Rawson,0,AR,1569455478,36,-43.30,-65.11,291.764,6.11,Complejo Bahia Ballenas,"-43.322365, -65.0501401",-43.3224,-65.0501
6,57,Pakdasht,0,IR,1569455478,16,35.48,51.68,293.150,2.10,Manzl Shahrdar,"35.5192946, 51.65088859999999",35.5193,51.6509
7,64,Jamestown,0,AU,1569455480,33,-33.21,138.60,292.685,3.63,Jamestown Hotel,"-33.2057381, 138.6035072",-33.2057,138.604
8,78,Saint-Joseph,0,FR,1569455483,72,43.56,6.97,293.150,1.50,Hôtel Barrière Le Majestic Cannes,"43.551077, 7.019327000000001",43.5511,7.01933
9,123,Hami,0,CN,1569455492,24,42.84,93.51,292.945,2.47,Hami Hotel,"42.8168371, 93.520449",42.8168,93.5204


In [84]:
# test = results[0]

# coord = test['results'][0]['geometry']['location']
# coord_lat = coord['lat']
# coord_lng = coord['lng']



# # pprint(test)

# print(coord)
# print(coord_lat)
# print(coord_lng)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [92]:
# Add marker layer ontop of heat map

mylocations = []

for x in range(10):
    
    place = (hotel_df.iloc[x,12], hotel_df.iloc[x,13])
    
    mylocations.append(place)
    
mylocations

hotelpins = gmaps.marker_layer(mylocations)

figure1.add_layer(hotelpins)

figure1

# Display Map



Figure(layout=FigureLayout(height='420px'))